<a href="https://colab.research.google.com/github/zmx110110/Emotion2Music/blob/main/musegan_emo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Musegan
!ls
# !unzip -n musegan-dev.zip
# !unzip -n JSB-Chorales-dataset-master.zip
%cd musegan-dev
!ls

/content/drive/MyDrive/Musegan
JSB-Chorales-dataset-master  musegan-dev  musegan-main
/content/drive/MyDrive/Musegan/musegan-dev
criterion.py  gan		     LICENSE	  requirements.txt
data	      images		     musegan.py   results
demo	      Jsb16thSeparated.npz   __pycache__  trainer.py
docs	      jsb-chorales-16th.pkl  README.md	  train.py


In [3]:
# !wget https://github.com/czhuang/JSB-Chorales-dataset/blob/master/Jsb16thSeparated.npz
# !wget https://github.com/czhuang/JSB-Chorales-dataset/blob/master/jsb-chorales-16th.pkl

In [4]:
!pip install progress

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for progress: filename=progress-1.6-py3-none-any.whl size=9611 sha256=6e19f900b7f61c8cb3e27a241f221800b551abb285b6b47143962454fca7f5cb
  Stored in directory: /root/.cache/pip/wheels/a2/68/5f/c339b20a41659d856c93ccdce6a33095493eb82c3964aac5a1
Successfully built progress


In [5]:
# !python train.py  --path /content/drive/MyDrive/Musegan/JSB-Chorales-dataset-master/Jsb16thSeparated.npz

In [6]:
%cd /content/drive/MyDrive/Musegan/musegan-dev/demo
import sys
sys.path.append("/content/drive/MyDrive/Musegan/musegan-dev/")

/content/drive/MyDrive/Musegan/musegan-dev/demo


In [7]:

import os
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch import nn

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [8]:
from gan import TemporalNetwork, BarGenerator, MuseCritic, MuseGenerator
tempnet = TemporalNetwork()
x = torch.rand(10, 32)
tempnet(x).shape
print('Number of parameters:', sum(p.numel() for p in tempnet.parameters()))

Number of parameters: 101472


In [9]:
bargenerator = BarGenerator()
a = torch.rand(10, 128)
bargenerator(a).shape
print('Number of parameters:', sum(p.numel() for p in bargenerator.parameters()))

Number of parameters: 1517313


In [10]:
generator = MuseGenerator()

cords = torch.rand(10, 32)
style = torch.rand(10, 32)
melody = torch.rand(10, 4, 32)
groove = torch.rand(10, 4, 32)

generator(cords, style, melody, groove).shape
print('Number of parameters:', sum(p.numel() for p in generator.parameters()))

Number of parameters: 6576612


In [11]:
critic = MuseCritic()
a = torch.rand(10, 4, 2, 16, 84)
critic(a).shape
print('Number of parameters:', sum(p.numel() for p in critic.parameters()))

Number of parameters: 1446401


In [12]:
from torch.utils.data import DataLoader
from data.utils import MidiDataset

dataset = MidiDataset(path='/content/drive/MyDrive/Musegan/JSB-Chorales-dataset-master/Jsb16thSeparated.npz')
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True)

In [13]:
generator = MuseGenerator(z_dimension=32, hid_channels=1024, hid_features=1024, out_channels=1).to(device)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=0.001, betas=(0.5, 0.9))

In [14]:
critic = MuseCritic(hid_channels=128,
                    hid_features=1024,
                    out_features=1).to(device)
c_optimizer = torch.optim.Adam(critic.parameters(), lr=0.001, betas=(0.5, 0.9))

In [15]:
from gan import initialize_weights

generator = generator.apply(initialize_weights)
critic = critic.apply(initialize_weights)


In [16]:
from trainer import Trainer

In [17]:
trainer = Trainer(generator, critic, g_optimizer, c_optimizer)

In [ ]:
trainer.train(dataloader, epochs=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

Epoch 0/1000 | Generator loss: 62.211 | Critic loss: -131.777
(fake: -262.063, real: 34.014, penalty: 96.271)
Epoch 10/1000 | Generator loss: 2108.767 | Critic loss: -153.933
(fake: -2130.307, real: 1882.094, penalty: 94.279)
Epoch 20/1000 | Generator loss: 1224.242 | Critic loss: -23.693
(fake: -1154.856, real: 1124.040, penalty: 7.122)
Epoch 30/1000 | Generator loss: 807.401 | Critic loss: -15.342
(fake: -591.403, real: 572.104, penalty: 3.957)
Epoch 40/1000 | Generator loss: 766.247 | Critic loss: -15.278
(fake: -1078.971, real: 1059.687, penalty: 4.006)
Epoch 50/1000 | Generator loss: 755.516 | Critic loss: -7.077
(fake: -448.050, real: 437.589, penalty: 3.383)
Epoch 60/1000 | Generator loss: -471.048 | Critic loss: -8.487
(fake: 220.095, real: -230.197, penalty: 1.616)
Epoch 70/1000 | Generator loss: -517.075 | Critic loss: -4.850
(fake: 422.215, real: -428.659, penalty: 1.593)
Epoch 80/1000 | Generator loss: 502.876 | Critic loss: -5.974
(fake: -691.667, real: 684.230, penalty: 1

In [ ]:
losses = trainer.data.copy()

import pandas as pd

df = pd.DataFrame.from_dict(losses)
#df.to_csv('results.csv', index=False)

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(losses['gloss'][:500], 'orange', label='generator')
plt.plot(losses['cfloss'][:500], 'r', label='critic fake')
plt.plot(losses['crloss'][:500], 'g', label='critic real')
plt.plot(losses['cploss'][:500], 'b', label='critic penalty')
plt.plot(losses['closs'][:500], 'm', label='critic')
plt.xlabel('epoch', fontsize=12)
plt.ylabel('loss', fontsize=12)
plt.grid()
plt.legend()
plt.show()
#plt.savefig('losses.png')

In [ ]:

generator = generator.eval().cpu()
#generator = torch.load('generator_e1000.pt')

## Inference模块

In [ ]:

generator = generator.eval().cpu()
#generator = torch.load('generator_e1000.pt')

In [ ]:

chords = torch.rand(1, 32)
style = torch.rand(1, 32)
melody = torch.rand(1, 4, 32)
groove = torch.rand(1, 4, 32)

preds = generator(chords, style, melody, groove).detach()


In [ ]:
from data.utils import postprocess
preds = preds.numpy()
music_data = postprocess(preds)


In [ ]:
filename = '1.midi'
music_data.write('midi', fp=filename)
!pwd